In [2]:
#TBfitting
from scipy.optimize import minimize
from numpy import *

class TbFitter():
	mu=0.
	T=1.
	M=False
	def fit(self,k,E,monitor=False,fitTol=None):
		self.M=monitor
		self.fitk=k
		self.fitE=E
		self.fitTol=fitTol
		gargs=self.material.getParameters()
		minimize(self.fitfun,gargs,tol=fitTol)
	def fitfun(self,args):
		self.material.setParameters(args)
		E=self.model.solve_all(self.fitk)
		diff=self.fitE-E
		diff=abs(diff)**2
		val=sum((diff*self.weightfun(self.fitE,E)).ravel())
		if self.M:
			print(val)
		self.lastval=val
		self.weightsum=sum((self.weightfun(self.fitE,E)).ravel())
		return val
	def weightfun(self,E1,E2):
		w1=1./cosh((E1-self.mu)/self.T)**2/self.T/8
		w2=1./cosh((E2-self.mu)/self.T)**2/self.T/8
		return w1+w2

In [4]:
#FitBulk.py

from pylab import *
from FCC_TB import *
from TBfitting import *
import bandReader as bReader

class fitmaterial(FCC_material):
	def getParameters(self):
		return self.parameters
	def setParameters(self,arg):
		self.parameters=arg
		self.update_all()

vkpoints, vbands, veferm=bReader.read("..\..\cdte.xml")
size=31
startband=24
endband=40

CdTe=FCC_conventional(0)
CdTe.material=fitmaterial(1)
CdTe.material.set_siteoperator(0,Es=5.77418741373, Ep=1.39447683813)
CdTe.material.set_interaction(0,0,Ess=-0.114171937589, Esp=0.25773764084, Exx=0.213075900579, Ezz=0.113264972856, Exy=0.31432951999)

fitter=TbFitter()
fitter.mu=4.
fitter.T=0.3
fitter.model=CdTe
fitter.material=CdTe.material
fitter.fit(vkpoints,transpose(vbands)[startband:endband],monitor=True,fitTol=0.01)
CdTe.material.printParameters()
print("minimum = "+str(fitter.lastval))
print("weight sum = "+str(fitter.weightsum))

side=ceil(size/2.)

k1=vkpoints[:side,:]
E1=vbands[:side,startband:endband]
k2=zeros([side,3])
E2=zeros([side,endband-startband])
ind=0
for i in arange(side):
	k2[i,:]=vkpoints[ind,:]
	E2[i,:]=vbands[ind,startband:endband]
	ind+=side-i
k3=zeros([side,3])
E3=zeros([side,endband-startband])
ind=0
for i in arange(side):
	k3[i]=vkpoints[ind,:]
	E3[i,:]=vbands[ind,startband:endband]
	ind+=(side-i)*(side-i+1)/2
k4=zeros([side,3])
E4=zeros([side,endband-startband])
ind=side-1
for i in arange(side):
	k4[i]=vkpoints[ind,:]
	E4[i,:]=vbands[ind,startband:endband]
	ind+=side-i-1

k=zeros((side-1)*4+1)
E=zeros([(side-1)*4+1,endband-startband])
k[0:side-1]=0.5-k3[side:0:-1,0]
E[0:side-1,:]=E3[side:0:-1,:]
k[side-1:2*(side-1)]=k1[0:side-1,0]+0.5
E[side-1:2*(side-1),:]=E1[0:side-1,:]
k[2*(side-1):3*(side-1)]=k4[0:side-1,1]+1.
E[2*(side-1):3*(side-1),:]=E4[0:side-1,:]
k[3*(side-1):4*(side-1)+1]=2.-k2[side::-1,1]
E[3*(side-1):4*(side-1)+1,:]=E2[side::-1,:]


for i in arange(endband-startband):
	plot(k[:],E[:,i],'*')

Emin=min(E.ravel())
Emax=max(E.ravel())


res=100
k=zeros([4*(res-1)+1,3])
kline=linspace(0,2,4*(res-1)+1)
k[:res,0]=k[:res,1]=k[:res,2]=linspace(0.5,0,res)
k[res-1:2*res-1,0]=linspace(0,0.5,res)
k[2*res-2:3*res-2,0]=linspace(0,0.5,res)
k[2*res-2:3*res-2,1]=0.5
k[3*res-3:4*res-3,0]=k[3*res-3:4*res-3,1]=linspace(0.5,0,res)

E=CdTe.solve_all(k)
for i in arange(E.shape[0]):
	plot(kline,E[i,:])

Emin=min(E.ravel())
Emax=max(E.ravel())
lh=array([floor(Emin),ceil(Emax)])
plot(0.5*ones(2),lh)
plot(1*ones(2),lh)
plot(1.5*ones(2),lh)

show()

ModuleNotFoundError: No module named 'FCC_TB'